In [1]:
#import modules and packages
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
#scrape a single game shots. We will allow users to enter a base url
base_url = 'https://understat.com/match/'
match = str(input('Please enter the match id: '))
url = base_url+match

Please enter the match id: 14700


In [3]:
#Use requests to get the webpage and BeautifulSoup to parse the page
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')

In [4]:
#get only the shotsData
strings = scripts[1].string

In [5]:
strings

"\n\tvar shotsData \t= JSON.parse('\\x7B\\x22h\\x22\\x3A\\x5B\\x7B\\x22id\\x22\\x3A\\x22409106\\x22,\\x22minute\\x22\\x3A\\x228\\x22,\\x22result\\x22\\x3A\\x22SavedShot\\x22,\\x22X\\x22\\x3A\\x220.8119999694824219\\x22,\\x22Y\\x22\\x3A\\x220.49900001525878906\\x22,\\x22xG\\x22\\x3A\\x220.059244439005851746\\x22,\\x22player\\x22\\x3A\\x22Ilkay\\x20G\\x5Cu00fcndogan\\x22,\\x22h_a\\x22\\x3A\\x22h\\x22,\\x22player_id\\x22\\x3A\\x22314\\x22,\\x22situation\\x22\\x3A\\x22OpenPlay\\x22,\\x22season\\x22\\x3A\\x222020\\x22,\\x22shotType\\x22\\x3A\\x22RightFoot\\x22,\\x22match_id\\x22\\x3A\\x2214700\\x22,\\x22h_team\\x22\\x3A\\x22Manchester\\x20City\\x22,\\x22a_team\\x22\\x3A\\x22Manchester\\x20United\\x22,\\x22h_goals\\x22\\x3A\\x220\\x22,\\x22a_goals\\x22\\x3A\\x222\\x22,\\x22date\\x22\\x3A\\x222021\\x2D03\\x2D07\\x2016\\x3A30\\x3A00\\x22,\\x22player_assisted\\x22\\x3A\\x22Riyad\\x20Mahrez\\x22,\\x22lastAction\\x22\\x3A\\x22Pass\\x22\\x7D,\\x7B\\x22id\\x22\\x3A\\x22409108\\x22,\\x22minute\\x22\

In [6]:
# strip unnecessary symbols and get only JSON data 
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
data = json.loads(json_data)

In [7]:
data

{'h': [{'id': '409106',
   'minute': '8',
   'result': 'SavedShot',
   'X': '0.8119999694824219',
   'Y': '0.49900001525878906',
   'xG': '0.059244439005851746',
   'player': 'Ilkay Gündogan',
   'h_a': 'h',
   'player_id': '314',
   'situation': 'OpenPlay',
   'season': '2020',
   'shotType': 'RightFoot',
   'match_id': '14700',
   'h_team': 'Manchester City',
   'a_team': 'Manchester United',
   'h_goals': '0',
   'a_goals': '2',
   'date': '2021-03-07 16:30:00',
   'player_assisted': 'Riyad Mahrez',
   'lastAction': 'Pass'},
  {'id': '409108',
   'minute': '18',
   'result': 'BlockedShot',
   'X': '0.8369999694824218',
   'Y': '0.585',
   'xG': '0.054408177733421326',
   'player': 'Gabriel Jesus',
   'h_a': 'h',
   'player_id': '5543',
   'situation': 'OpenPlay',
   'season': '2020',
   'shotType': 'RightFoot',
   'match_id': '14700',
   'h_team': 'Manchester City',
   'a_team': 'Manchester United',
   'h_goals': '0',
   'a_goals': '2',
   'date': '2021-03-07 16:30:00',
   'player_a

In [8]:
x = []
y = []
xG = []
result = []
team = []
data_away = data['a']
data_home = data['h']

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])


In [9]:
col_names = ['x','y','xG','result','team']
df = pd.DataFrame([x,y,xG,result,team],index=col_names)
df = df.T

In [10]:
df

,x,y,xG,result,team
0,0.8119999694824219,0.49900001525878906,0.059244439005851746,SavedShot,Manchester City
1,0.8369999694824218,0.585,0.054408177733421326,BlockedShot,Manchester City
2,0.82,0.610999984741211,0.047269511967897415,MissedShots,Manchester City
3,0.7669999694824219,0.710999984741211,0.026997065171599388,BlockedShot,Manchester City
4,0.759000015258789,0.5570000076293945,0.02475670725107193,SavedShot,Manchester City
5,0.7619999694824219,0.4329999923706055,0.025984913110733032,MissedShots,Manchester City
6,0.86,0.664000015258789,0.0456041619181633,BlockedShot,Manchester City
7,0.7519999694824219,0.385,0.017544662579894066,BlockedShot,Manchester City
8,0.8969999694824219,0.44,0.016498293727636337,BlockedShot,Manchester City
9,0.845,0.5029999923706054,0.05971451848745346,SavedShot,Manchester City
